# Utils

**Импорты библиотек:**

In [1]:
!pip install -q tqdm==4.66.5
!pip install -q torch==2.4.1+cu121
!pip install -q torchvision==0.19.1+cu121
!pip install -q transformers==4.44.2
!pip install -q comet_ml==3.47.6
!pip install -q triton==3.1.0
!pip install -q torchattacks==3.5.1
!pip install -q jax==0.4.38
!pip install -q jaxlib==0.4.38
!pip install -q torchlars==0.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dep

In [2]:
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os

import comet_ml

import torch
import torchvision
from torchvision.transforms import v2
import torchattacks
from torchlars import LARS

**Utils:**

**Функция для фиксирования сида:**

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Additional utils:**

In [4]:
def get_root(env):
    # supports env == "colab", env == "kaggle"
    if env == "kaggle":
        return "/kaggle/working/"
    else:
        return "./"

In [5]:
# this augmentation first converts PIL image to Tensor,
# then casts resulting tensor to type=torch.float32,
# finally normalizes the image

default_aug = v2.Compose([v2.PILToTensor(),
                          v2.ToDtype(torch.float32, scale=True),
                          v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [6]:
# This function is used later in DataLoaders to fix the seed setting bugs when using several workers

def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed() % (2**32)
    random.seed(torch_seed)
    np.random.seed(torch_seed)

**Датасет + DataLoaders:**

In [7]:
def get_CIFAR10_data(train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    print("Prepare Test Set:  ✅")
    return data_train, data_test

In [8]:
def get_CIFAR10_dataloaders(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Test Set:  ✅")
    return dataloader_train, dataloader_test

**Код цикла предобучения/дообучения SSL-метода:**

In [9]:
class ViewGenerator(object):
    # Take several copies of one image and apply a transform to them
    def __init__(self, transform, n_views=2):
        self.transform = transform
        self.n_views = n_views
        
    def __call__(self, x):
        return [self.transform(x) for i in range(self.n_views)]

In [10]:
def get_CIFAR10_contrastive_dataloaders(batch_size, transform=default_aug, env="colab"):
    root = get_root(env)
    data = torchvision.datasets.CIFAR10(root=root,
                                        train=True, 
                                        transform=ViewGenerator(transform=transform, n_views=2),
                                        download=True)
    dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Set: ✅")
    return dataloader

In [11]:
class SimCLR(torch.nn.Module):
    def __init__(self, encoder, head_out_dim, head_hidden_dim, enc_out_dim=512):
        super(SimCLR, self).__init__()
        self.encoder = encoder
        # MLP projection head (authors of SimCLR show that it is better to use MLP rather than simple Linear layer as a head)
        self.head = torch.nn.Sequential(
            torch.nn.Linear(enc_out_dim, head_hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(head_hidden_dim, head_out_dim)
        )

    def forward(self, x):
        enc_features = self.encoder(x)
        logits = self.head(enc_features)
        return logits

In [12]:
def InfoNCE_loss(logits, temperature=0.2):
    cosine_sim = torch.nn.functional.cosine_similarity(torch.unsqueeze(logits, dim=1), torch.unsqueeze(logits, dim=0), dim=-1)
    
    # Mask out cosine similarity to itself
    mask = torch.eye(cosine_sim.shape[0], dtype=torch.bool, device=logits.device)
    cosine_sim.masked_fill_(mask, -torch.inf)
    
    # Find positive example - it is (batch_size / 2) positions away from the original example
    positive_mask = mask.roll(shifts=cosine_sim.shape[0] // 2, dims=0)
    
    # InfoNCE loss
    cosine_sim = cosine_sim / temperature
    loss = -cosine_sim[positive_mask] + torch.logsumexp(cosine_sim, dim=-1)
    loss = loss.mean()
    return loss

In [13]:
def pretrain_epoch(model, optimizer, loader, scaler, epoch, exp):
    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, _) in enumerate(tqdm(loader)):
        data = torch.cat(data, dim=0)
        data = data.to(device)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = InfoNCE_loss(logits)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    return scaler


def pretrain(model, exp_name, optimizer, epochs, pretrain_loader, warmup_scheduler=None, scheduler=None, warmup_epochs=0, exp=None, env="colab"):
    scaler = torch.GradScaler("cuda")
    
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        scaler = pretrain_epoch(model, optimizer, pretrain_loader, scaler, epoch, exp)
        
        if warmup_scheduler is not None and epoch <= warmup_epochs:
            warmup_scheduler.step()
            if epoch == warmup_epochs:
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(epochs - warmup_epochs), eta_min=1e-4)
        else:
            if scheduler is not None:
                scheduler.step()

    filepath = f"{get_root(env)}weights_{exp_name}_epoch{epochs}.pth"
    torch.save(model.state_dict(), filepath)
    print("Save State of the Best Model: ✅")

    if exp is not None:
        exp.log_model(name=filepath[len(get_root(env)):-4],
                      file_or_folder=filepath)

In [24]:
comet_ml.login()

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
SimCLR_aug = v2.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.RandomResizedCrop(
        size=(32, 32),
        scale=(0.08, 1.0),
        interpolation=torchvision.transforms.InterpolationMode.BICUBIC,
        antialias=True),
    v2.RandomApply(
        torch.nn.ModuleList([
            v2.ColorJitter(
                brightness=(0.6, 1.4),
                contrast=(0.6, 1.4),
                saturation=(0.6, 1.4),
                hue=(-0.1, 0.1)
            )
        ]),
        p=0.8
    ),
    v2.RandomGrayscale(p=0.2),
    v2.RandomApply(
        torch.nn.ModuleList([
            v2.GaussianBlur(kernel_size=3)
        ]),
        p=0.5
    ),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [17]:
NUM_EPOCHS = 10
NUM_WARMUP_EPOCHS = 2
BATCH_SIZE = 256
exp_name = "SimCLR_Pretrain"

In [18]:
# random seed
seed_everything(0)

# get data and batchify it
pretrain_loader = get_CIFAR10_contrastive_dataloaders(batch_size=BATCH_SIZE, transform=SimCLR_aug, env="colab")

# initialize encoder
encoder = torchvision.models.resnet.resnet18()
encoder.fc = torch.nn.Identity()
encoder.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
encoder.maxpool = torch.nn.Identity()
torch.nn.init.kaiming_normal_(encoder.conv1.weight, mode="fan_out", nonlinearity="relu") # initialize conv1 weights the same way as all the other Conv2d weights in ResNet18

# initialize model
model = SimCLR(encoder=encoder, head_out_dim=256, head_hidden_dim=2048)
model.to(device)

# choose optimizer and scheduler (if needed)
base_optimizer = torch.optim.SGD(model.parameters(), lr=0.4, weight_decay=1e-4)
optimizer = LARS(optimizer=base_optimizer, eps=1e-8, trust_coef=0.02)

warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.33, total_iters=NUM_WARMUP_EPOCHS)

100%|██████████| 170498071/170498071 [00:01<00:00, 102921845.16it/s]


Extracting ./cifar-10-python.tar.gz to ./
Prepare Set: ✅


In [19]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

pretrain(model, exp_name, optimizer, NUM_EPOCHS, pretrain_loader, warmup_scheduler=warmup_scheduler, scheduler=None, warmup_epochs=NUM_WARMUP_EPOCHS, exp=exp, env="colab")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/6957af7f9f9d4667a2844337a9b0363f



Epoch 1:



  0%|          | 0/196 [00:00<?, ?it/s]COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
/usr/local/lib/python3.10/dist-packages/torchlars/lars.py:140: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1581.)
  p.grad.add_(weight_decay, p.data)
100%|██████████| 196/196 [01:15<00:00,  2.59it/s]


Epoch 2:



100%|██████████| 196/196 [01:13<00:00,  2.67it/s]


...

100%|██████████| 196/196 [01:14<00:00,  2.64it/s]


Epoch 10:



100%|██████████| 196/196 [01:14<00:00,  2.65it/s]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : SimCLR_Pretrain
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/6957af7f9f9d4667a2844337a9b0363f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                   : (106301.21875, 403316.96875)
COMET INFO:     train loss (by steps) [1960] : (2.174712896347046, 6.18222188949585)
COMET INFO:   Others:
COMET INFO:     Name : SimCLR_Pretrain
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model-element       : 1 (48.69 MB)
COMET INFO:    

Save State of the Best Model: ✅


COMET INFO: Please wait for metadata to finish uploading (timeout is 3600 seconds)
COMET INFO: Uploading 33 metrics, params and output messages
COMET INFO: Please wait for assets to finish uploading (timeout is 10800 seconds)
COMET INFO: Still uploading 1 file(s), remaining 17.19 MB/48.69 MB


**Код для обычного/adversarial Fine-Tuning'а (на одном GPU):**

In [14]:
def eval_epoch(model, loader, attack=None):
    val_loss = 0.
    val_acc = 0.

    model.eval()

    for data, target in tqdm(loader):
        data = data.to(device)
        target = target.to(device)

        if attack is not None:
            # perform adversarial attack
            with torch.enable_grad():
                data = attack(data, target)
        
        logits = model(data)
        loss = torch.nn.functional.cross_entropy(logits, target)

        val_loss += (loss.item() * data.shape[0])
        val_acc += (torch.argmax(logits, dim=1) == target).sum().item()

    val_loss /= len(loader.dataset)
    val_acc /= len(loader.dataset)
    return val_loss, val_acc


def train_epoch(model, optimizer, loader, scaler, epoch, exp, is_attack_used=False):
    train_loss = 0.
    train_acc = 0.

    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, target) in enumerate(tqdm(loader)):
        data = data.to(device)
        target = target.to(device)

        if is_attack_used != "no":
            if is_attack_used == "BIM":
                attack = torchattacks.BIM(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            else:
                attack = torchattacks.PGD(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            # perform adversarial attack
            data = attack(data, target)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = torch.nn.functional.cross_entropy(logits, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            train_loss += loss.item() * data.shape[0]
            train_acc += (torch.argmax(logits, dim=1) == target).sum().item()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    train_loss /= len(loader.dataset)
    train_acc /= len(loader.dataset)
    return train_loss, train_acc, scaler


def train(model, exp_name, optimizer, epochs, train_loader, val_loader, is_train_attack_used="no", scheduler=None, exp=None, env="colab"):
    best_val_acc = 0.
    best_val_acc_epoch = None
    best_model_state_filepath = None

    val_attacks_names = ["No Attack"]

    scaler = torch.GradScaler("cuda")

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        metrics = dict()
        
        train_loss, train_acc, scaler = train_epoch(model, optimizer, train_loader, scaler, epoch, exp, is_attack_used=is_train_attack_used)

        val_attacks = [None]
        
        for it in range(len(val_attacks)):
            val_loss, val_acc = eval_epoch(model, val_loader, attack=val_attacks[it])
            metrics[val_attacks_names[it]] = {"Loss": val_loss, "Accuracy": val_acc}

        if scheduler is not None:
            scheduler.step()

        print("\n")
        print(" Train:")
        print(f"  Loss: {train_loss}, Accuracy: {train_acc}")
        print(" Validation:")
        for it in metrics.items():
            print(f"  Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


        if exp is not None:
            exp.log_metrics(
                   {"train loss": train_loss,
                    "train accuracy": train_acc},
                    step=(epoch * len(train_loader)))
            for it in metrics.items():
                exp.log_metrics(
                    {f"validation loss ({it[0]})": it[1]['Loss'],
                     f"validation accuracy ({it[0]})": it[1]['Accuracy']},
                     step=(epoch * len(train_loader)))

        val_acc = metrics["No Attack"]["Accuracy"]
        
        if val_acc > best_val_acc:
            print("🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉")
            best_val_acc = val_acc
            best_val_acc_epoch = epoch
            filepath = f"{get_root(env)}best_weights_{exp_name}_epoch{best_val_acc_epoch}.pth"
            torch.save(model.state_dict(), filepath)
            print("Save State of the Best Model: ✅")
            if best_model_state_filepath is not None:
                os.remove(best_model_state_filepath)
                print("Delete State File of the Previous Best Model: ✅")
            best_model_state_filepath = filepath

        print('-----------------------------------------------------------------\n')

    if exp is not None:
        exp.log_model(name=best_model_state_filepath[len(get_root(env)):-4],
                      file_or_folder=best_model_state_filepath)


def test(model, loader, env="colab"):
    attacks_names = ["No Attack", "i-FGSM", "PGD", "DI-FGSM"]
    
    metrics = dict()

    attacks = [None, 
               torchattacks.BIM(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.DIFGSM(model, eps=(8 / 255), alpha=(2 / 255), steps=10, resize_rate=0.9, diversity_prob=0.5)
              ]
    
    for it in range(len(attacks)):
        loss, acc = eval_epoch(model, loader, attack=attacks[it])
        metrics[attacks_names[it]] = {"Loss": loss, "Accuracy": acc}

    print("\n")
    print("Final Metrics:")
    for it in metrics.items():
        print(f" {it[0]}:")
        print(f"   Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


# Эксперименты

**Downstream-модель:**

In [15]:
class DownstreamModel(torch.nn.Module):
    def __init__(self, encoder, head):
        super(DownstreamModel, self).__init__()
        self.encoder = encoder
        self.head = head

    def forward(self, x):
        enc_features = self.encoder(x)
        logits = self.head(enc_features)
        return logits

**Берём энкодер из модели, прикрепляем линейную голову:**

In [22]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# initialize SimCLR encoder
encoder = torchvision.models.resnet.resnet18()
encoder.fc = torch.nn.Identity()
encoder.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
encoder.maxpool = torch.nn.Identity()
torch.nn.init.kaiming_normal_(encoder.conv1.weight, mode="fan_out", nonlinearity="relu") # initialize conv1 weights the same way as all the other Conv2d weights in ResNet18

# initialize SimCLR
model = SimCLR(encoder=encoder, head_out_dim=256, head_hidden_dim=2048)
model.to(device)

model_state_dict = torch.load("./weights_SimCLR_Pretrain_epoch10.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.encoder

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 512 # output size of SimCLR encoder

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.ReLU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head)
model.to(device)

DownstreamModel(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
    

**SimCLR (SFT):**

In [23]:
comet_ml.login()

In [25]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "SimCLR_SFT"

In [27]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [28]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="no", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/86f5bfad44944f0385bec4ae66a7f2b3

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:03<00:00, 23.16it/s]




 Train:
  Loss: 0.9103827179336548, Accuracy: 0.68264
 Validation:
  Loss: 0.7328914521217346, Accuracy: 0.7403
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:03<00:00, 24.57it/s]


...



 Train:
  Loss: 0.18254395937919618, Accuracy: 0.94266
 Validation:
  Loss: 0.6022506805419922, Accuracy: 0.8108
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------

Epoch 5:



100%|██████████| 79/79 [00:03<00:00, 25.06it/s]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : SimCLR_SFT
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/86f5bfad44944f0385bec4ae66a7f2b3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (3324.70068359375, 150592.5)
COMET INFO:     train accuracy [5]                  : (0.68264, 0.9776)
COMET INFO:     train loss (by steps) [1955]        : (0.03385128080844879, 2.2978591918945312)
COMET INFO:     train loss [5]                      : (0.08941236767768859, 0.9103827179336548)
COMET INFO:     validation accuracy (No Attack) [5] : (0.7403, 0.8183)
COMET INFO:     vali



 Train:
  Loss: 0.08941236767768859, Accuracy: 0.9776
 Validation:
  Loss: 0.5914972339630127, Accuracy: 0.8183
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------




100%|██████████| 79/79 [01:36<00:00,  1.23s/it]



Final Metrics:
 No Attack:
   Loss: 0.5914972339630127, Accuracy: 0.8183
 i-FGSM:
   Loss: 5.551533029174805, Accuracy: 0.2349
 PGD:
   Loss: 10.571024765014648, Accuracy: 0.011
 DI-FGSM:
   Loss: 8.680614855957032, Accuracy: 0.0226


**SimCLR (Adversarial SFT with i-FGSM):**

In [29]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# initialize SimCLR encoder
encoder = torchvision.models.resnet.resnet18()
encoder.fc = torch.nn.Identity()
encoder.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
encoder.maxpool = torch.nn.Identity()
torch.nn.init.kaiming_normal_(encoder.conv1.weight, mode="fan_out", nonlinearity="relu") # initialize conv1 weights the same way as all the other Conv2d weights in ResNet18

# initialize SimCLR
model = SimCLR(encoder=encoder, head_out_dim=256, head_hidden_dim=2048)
model.to(device)

model_state_dict = torch.load("./weights_SimCLR_Pretrain_epoch10.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.encoder

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 512 # output size of SimCLR encoder

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.ReLU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head)
model.to(device)

DownstreamModel(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
    

In [30]:
comet_ml.login()

In [31]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "SimCLR_Adversarial_SFT_BIM"

In [33]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [34]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/4bf1baee57d9455c85f0fc820edb8469

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:03<00:00, 25.46it/s]




 Train:
  Loss: 1.0919768834114074, Accuracy: 0.60842
 Validation:
  Loss: 0.7880367339134217, Accuracy: 0.716
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:03<00:00, 25.73it/s]


...

100%|██████████| 79/79 [00:03<00:00, 25.67it/s]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : SimCLR_Adversarial_SFT_BIM
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/4bf1baee57d9455c85f0fc820edb8469
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (10466.81640625, 152427.0)
COMET INFO:     train accuracy [5]                  : (0.60842, 0.90786)
COMET INFO:     train loss (by steps) [1955]        : (0.15724311769008636, 2.3258514404296875)
COMET INFO:     train loss [5]                      : (0.2816477269935608, 1.0919768834114074)
COMET INFO:     validation accuracy (No Attack) [5] : (0.716, 0.817)
COMET IN



 Train:
  Loss: 0.2816477269935608, Accuracy: 0.90786
 Validation:
  Loss: 0.5391895300388336, Accuracy: 0.817
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: Please wait for assets to finish uploading (timeout is 10800 seconds)
COMET INFO: Still uploading 1 file(s), remaining 40.50 MB/44.73 MB


100%|██████████| 79/79 [01:36<00:00,  1.22s/it]



Final Metrics:
 No Attack:
   Loss: 0.5391895300388336, Accuracy: 0.817
 i-FGSM:
   Loss: 0.9786480871200561, Accuracy: 0.6808
 PGD:
   Loss: 3.3085753410339356, Accuracy: 0.1516
 DI-FGSM:
   Loss: 3.1250150234222414, Accuracy: 0.1745


**SimCLR (Adversarial SFT with PGD):**

In [17]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# initialize SimCLR encoder
encoder = torchvision.models.resnet.resnet18()
encoder.fc = torch.nn.Identity()
encoder.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
encoder.maxpool = torch.nn.Identity()
torch.nn.init.kaiming_normal_(encoder.conv1.weight, mode="fan_out", nonlinearity="relu") # initialize conv1 weights the same way as all the other Conv2d weights in ResNet18

# initialize SimCLR
model = SimCLR(encoder=encoder, head_out_dim=256, head_hidden_dim=2048)
model.to(device)

model_state_dict = torch.load("./weights_SimCLR_Pretrain_epoch10.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.encoder

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 512 # output size of SimCLR encoder

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.ReLU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head)
model.to(device)

DownstreamModel(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
    

In [19]:
comet_ml.login()

In [20]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
NUM_EPOCHS = 5
BATCH_SIZE = 128
exp_name = "SimCLR_Adversarial_SFT_PGD"

In [22]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

100%|██████████| 170498071/170498071 [00:02<00:00, 81050869.95it/s]


Extracting ./cifar-10-python.tar.gz to ./
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [23]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/626eb124f1d14efd9e1291e5daa361c0



Epoch 1:



100%|██████████| 79/79 [00:03<00:00, 25.30it/s]




 Train:
  Loss: 1.512427154006958, Accuracy: 0.44314
 Validation:
  Loss: 4.736591479492187, Accuracy: 0.2384
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:03<00:00, 25.63it/s]


...

100%|██████████| 79/79 [00:03<00:00, 25.60it/s]
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : SimCLR_Adversarial_SFT_PGD
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/626eb124f1d14efd9e1291e5daa361c0
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [196]                          : (41399.59375, 153568.5)
COMET INFO:     train accuracy [5]                  : (0.44314, 0.69908)
COMET INFO:     train loss (by steps) [1955]        : (0.6170915365219116, 2.3432693481445312)
COMET INFO:     train loss [5]                      : (0.8292864834976196, 1.512427154006958)
COMET INFO:     validation accuracy (No Attack) [5] : (0.2384, 0.3925)
COMET INFO:



 Train:
  Loss: 0.8292864834976196, Accuracy: 0.69908
 Validation:
  Loss: 3.875953694152832, Accuracy: 0.3524
-----------------------------------------------------------------



COMET INFO: Please wait for assets to finish uploading (timeout is 10800 seconds)
COMET INFO: All assets have been sent, waiting for delivery confirmation


100%|██████████| 79/79 [01:37<00:00,  1.23s/it]



Final Metrics:
 No Attack:
   Loss: 3.875953694152832, Accuracy: 0.3524
 i-FGSM:
   Loss: 4.3787047439575195, Accuracy: 0.2859
 PGD:
   Loss: 1.6716163145065308, Accuracy: 0.4218
 DI-FGSM:
   Loss: 1.5476438034057618, Accuracy: 0.4613
